In [1]:
# olmocr python 가상환경 설정

In [2]:
# !pip install -q nest_asyncio

In [3]:
import nest_asyncio
#nest_asyncio는 Jupyter Notebook(ipynb)에서 비동기(async/await) 코드를 실행하기 위해 필요
nest_asyncio.apply()

## OLMOCR 설치

In [4]:
# RestFul서비스를 구축하고 테스트하기 위해 필요
# !pip install -Uq olmocr openai ipywidgets 

In [5]:
# !pip install transformers==4.49.0

In [6]:
# !git clone https://github.com/allenai/olmocr.git
# %cd olmocr
# !pip install -e .

## PDF 테스트 파일 설정

In [7]:
import asyncio
import json
from pathlib import Path
from tqdm import tqdm
import os

from pypdf import PdfReader
from olmocr.pipeline import build_page_query, build_dolma_document, PageResult
from olmocr.prompts import PageResponse

from olmocr.version import VERSION
from glob import glob

from openai import OpenAI
client = OpenAI(base_url="http://localhost:8000/v1", api_key="token-abc123", timeout=120)

async def process_page(filename, page_num):
    query = await build_page_query(filename,
                            page=page_num,
                            target_longest_image_dim=1024,
                            target_anchor_text_len=6000)
    
    # query['model'] = 'Qwen2.5-VL-7B-Instruct'
    query['model'] = 'olmOCR-7B-0225-preview'

    # print(query)
    response = client.chat.completions.create(**query)

    content = response.choices[0].message.content

    page_response = PageResponse(
        primary_language = "ko",
        is_rotation_valid = False,
        rotation_correction = 0,
        is_table = False,
        is_diagram = False,
        natural_text = content
    )

    return PageResult(
        filename,
        page_num,
        page_response,
        input_tokens=response.usage.prompt_tokens,
        output_tokens=response.usage.completion_tokens,
        is_fallback=False
    )

In [8]:
# async def get_content(file_path):
#     reader = PdfReader(file_path)
#     file_name = Path(file_path).stem
#     num_pages = reader.get_num_pages()
#     results = []
    
#     for page_num in trange(1, num_pages + 1):
#         result = await process_page(file_path, page_num)
        
#         results.append(result)
        
#         print(result.response)
        
#         print("--------------------------------")
#         # 만약 result.response가 JSON 문자열이라면
#         if isinstance(result.response.natural_text, str):
#             try:
#                 response_dict = json.loads(result.response.natural_text)
#                 print(f"Processed: {response_dict['natural_text']}")
#             except json.JSONDecodeError:
#                 print(f"Processed: {result.response.natural_text}")
#         else:
#             # PageResponse 객체라면
#             print(f"Processed: {result.response.natural_text}")
            
#     print("--- Get PDF Results ---")
#     text_parts = []
#     for result in results:
#         if isinstance(result.response.natural_text, str):
#             try:
#                 response_dict = json.loads(result.response.natural_text)
#                 text_parts.append(response_dict['natural_text'])
#             except json.JSONDecodeError:
#                 # JSON 파싱에 실패하면 원본 텍스트 추가
#                 text_parts.append(result.response.natural_text)
#         else:
#             text_parts.append(result.response.natural_text)
    
#     text = "\n".join(text_parts)
#     print(text)

#     # 텍스트를 마크다운 파일로 저장
#     output_folder = Path("./results")
#     output_folder.mkdir(exist_ok=True, parents=True)
    
#     md_path = output_folder.joinpath(file_name).with_suffix(".md")
#     full_path = md_path.resolve()
#     print(f"마크다운 저장 경로: {full_path}")
#     with open(full_path, "w", encoding="utf-8") as file:
#         file.write(text)

#     # build_dolma_document 부분 주석 처리
#     # print("--- Build the dolma results ---")
#     # doc = build_dolma_document(file_path, results)
#     # pdf_doc = json.dumps(doc, ensure_ascii=False)
#     # print(pdf_doc)
#     # dolma_path = output_folder.joinpath(file_name).with_suffix(".results.json")
#     # full_path = dolma_path.resolve()
#     # print(full_path)
#     # with open(full_path, "w", encoding="utf-8") as file:
#     #     file.write(pdf_doc + "\n")

In [9]:
async def get_content(file_path, single_page=False, page_num=None):
    reader = PdfReader(file_path)
    file_name = Path(file_path).stem
    num_pages = reader.get_num_pages()
    results = []
    
    # 파일명에 대한 폴더 생성
    output_folder = Path("./results")
    output_folder.mkdir(exist_ok=True, parents=True)
    
    pdf_folder = output_folder.joinpath(file_name)
    
    # 폴더가 이미 존재하는지 확인
    if pdf_folder.exists() and len(os.listdir(pdf_folder)) == num_pages and not single_page:
        print(f"폴더 '{pdf_folder}'가 이미 존재합니다. 처리를 건너뜁니다.")
        return  # 함수 종료
    
    pdf_folder.mkdir(exist_ok=True, parents=True)
    
    page_num = 1 if page_num is None else page_num
    pbar = tqdm(total=num_pages if not single_page else 1)
    
    while page_num <= num_pages:
        result = await process_page(file_path, page_num)

        print("--------------------------------")
        
        # 페이지 텍스트 추출
        page_text = None
        if isinstance(result.response.natural_text, str):
            try:
                response_dict = json.loads(result.response.natural_text)
                page_text = response_dict.get('natural_text')
                # print(f"Processed: {page_text}")
            except json.JSONDecodeError:
                print(f"Error: JSONDecodeError")                
        else:
            page_text = result.response.natural_text
            # print(f"Processed: {page_text}")
        
        # None이거나 빈 문자열이면 재시도
        if page_text is None or page_text == "":
            print(f"페이지 {page_num}의 텍스트가 없습니다. 재시도합니다.")
            # 잠시 대기 후 재시도
            await asyncio.sleep(1)
            continue  # 같은 페이지 다시 시도
        
        # 성공적으로 텍스트를 얻었으면 결과 저장 및 다음 페이지로
        results.append(result)
        
        # 페이지별 마크다운 파일 저장
        page_md_path = pdf_folder.joinpath(f"page_{page_num}.md")
        with open(page_md_path, "w", encoding="utf-8") as file:
            file.write(page_text)
        
        if single_page:
            break
        
        # 다음 페이지로 이동
        page_num += 1
        pbar.update(1)
    
    print("---End---")
    pbar.close()

In [10]:
def main():
    file_dir = "data/건설안전지침/"
        
    files = glob(file_dir + "*.pdf")
    
    for file in files:
        print(f"Processing {Path(file).stem}")
        # Run the async event loop
        asyncio.run(get_content(file))

### 특정 문서 / 페이지 디버깅을 위한 함수

In [11]:
# def main():
#     files = ["data/건설안전지침/재사용 가설기자재 성능기준에 관한 지침.pdf"]
    
#     for file in files:
#         print(f"Processing {Path(file).stem}")
#         # Run the async event loop
        
#         for page_num in [23, 25, 28, 30, 33, 37, 42, 43, 44, 45, 48, 49]:
#             asyncio.run(get_content(file, single_page=True, page_num=page_num))
#         # asyncio.run(get_content(file, single_page=True, page_num=14))

In [ ]:
if __name__ == "__main__":
    main()

Processing 건설공사의 고소작업대 안전보건작업지침
폴더 'results/건설공사의 고소작업대 안전보건작업지침'가 이미 존재합니다. 처리를 건너뜁니다.
Processing 취약시기 건설현장 안전작업지침
폴더 'results/취약시기 건설현장 안전작업지침'가 이미 존재합니다. 처리를 건너뜁니다.
Processing 수직형 추락방망 설치 기술지침
폴더 'results/수직형 추락방망 설치 기술지침'가 이미 존재합니다. 처리를 건너뜁니다.
Processing 초고층 건축물공사(일반사항) 안전보건작업지침
폴더 'results/초고층 건축물공사(일반사항) 안전보건작업지침'가 이미 존재합니다. 처리를 건너뜁니다.
Processing 타일(Tile) 공사 안전보건작업 지침
폴더 'results/타일(Tile) 공사 안전보건작업 지침'가 이미 존재합니다. 처리를 건너뜁니다.
Processing 터널공사(Shield-T.B.M공법) 안전보건작업 지침
폴더 'results/터널공사(Shield-T.B.M공법) 안전보건작업 지침'가 이미 존재합니다. 처리를 건너뜁니다.
Processing 현수교 주탑시공 안전보건작업지침
폴더 'results/현수교 주탑시공 안전보건작업지침'가 이미 존재합니다. 처리를 건너뜁니다.
Processing 블록식 보강토 옹벽 공사 안전보건작업 지침
폴더 'results/블록식 보강토 옹벽 공사 안전보건작업 지침'가 이미 존재합니다. 처리를 건너뜁니다.
Processing 항타기, 항발기 사용 작업계획서 작성지침
폴더 'results/항타기, 항발기 사용 작업계획서 작성지침'가 이미 존재합니다. 처리를 건너뜁니다.
Processing 조적공사 안전보건작업 기술지침
폴더 'results/조적공사 안전보건작업 기술지침'가 이미 존재합니다. 처리를 건너뜁니다.
Processing 시스템 동바리 안전작업 지침
폴더 'results/시스템 동바리 안전작업 지침'가 이미 존재합니다. 처리를 건너뜁니다.
Processing 야간 건설공사 안전보건작업 지침
폴더 

  3%|▎         | 1/32 [00:04<02:20,  4.54s/it]

--------------------------------
Processed: PCT거더 교량공사 안전보건작업 지침


INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
  6%|▋         | 2/32 [00:12<03:13,  6.46s/it]

--------------------------------
Processed: 안전보건작업지침의 개요

○ 작성자 : 한국안전학회 백신원

○ 제·개정 경과

- 2013년 9월 건설안전분야 제정위원회 심의(제정)

○ 관련규격 및 자료

- 최신 교량공학(동명사, 황학주)
- 도로교 표준시방서(대한토목학회)
- 교량공사(PCT거더공법) 시공자료
- KS D ISO 10721-2 : 강구조물-제2부 : 조립 및 가설

○ 관련법규 · 규칙 · 고시 등

- 산업안전보건기준에 관한 규칙 제379조~제393조

○ 작업지침의 적용 및 문의

이 작업지침에 대한 의견 또는 문의는 한국산업안전보건공단 홈페이지 안전보건기술지침 소관 분야별 문의처 안내를 참고하시기 바랍니다.

공표일자 : 2013년 10월 2일
제 정 자 : 한국산업안전보건공단 이사장


...

--------------------------------
Processed: 옹벽(콘크리트 옹벽)공사의 안전보건작업지침

1. 목 적

이 지침은 산업안전보건기준에 관한 규칙에 의거 옹벽공사(콘크리트 옹벽) 중 발생할 수 있는 토사의 무너짐, 장비에 의한 부딪힘 및 깔림, 고소작업시 떨어짐 등의 재해를 예방할 수 있는 콘크리트 옹벽공사 안전보건 작업지침을 정함을 목적으로 한다.

2. 적용범위

이 지침은 건설현장에서 시공되고 있는 옹벽공사 중 콘크리트 옹벽에 대하여 적용한다.

3. 용어의 정의

(1) 이 지침에서 사용되는 용어의 뜻은 다음과 같다.

(가) “옹벽”이라 함은 토압에 저항하는 가장 일반적인 구조물로 용지의 제한에 따른 토지의 최적 이용을 목적으로 설치하는 구조물을 말한다.


INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
 27%|██▋       | 4/15 [00:34<01:44,  9.52s/it]

--------------------------------
Processed: (나) “콘크리트 옹벽”이란 절토 또는 성토 비탈면이 흙의 압력으로 인해 붕괴 되는 것을 방지할 목적으로 설치한 콘크리트 벽체 구조물을 말한다.

(다) “시공이음”이라 함은 이미 시공된 콘크리트에 새로운 콘크리트를 쳐서 잇기 위해 만든 이음말을 말한다.

(라) “뒤채움”이라 함은 구조물 뒷면에 잡석·자갈·콘크리트 등을 채우는 것으로 구조물의 안정을 도모하는 동시에 배면의 배수를 용이하게 작용을 말한다.

(마) “배수공”이라 함은 옹벽의 배면에 모인 우수 및 침투수를 배출시키기 위한 배수구조물로 말한다.

(바) “래머(Rammer)”라 함은 동력에 의해 다짐판을 진동시켜 지반의 중와 충격에 의해 지반을 다지는 기계이며 <그림 2>와 같다.

(사) “콤팩터(compactor)”라 함은 뒷채움 토사에 하중을 가하여 진동, 충격을 줄이므로써 보다 무거운 하중에 견딜 수 있도록 지반을 다지기 위한 기계이며 <그림 3>과 같다.

(2) 그 밖에 이 지침에서 사용하는 용어의 정의는 이 지침에서 특별히 규정하는 경우를 제외하고는 산업안전보건법 및 산업안전보건법 시행령, 산업안전보건법 시행규칙, 산업안전보건에 관한 규칙 및 고시에서 정하는 바에 따른다.


INFO:httpx:HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"
 33%|███▎      | 5/15 [00:43<01:30,  9.05s/it]

--------------------------------
Processed: 4. 콘크리트 용벽의 종류

(1) 콘크리트 용벽의 종류는 다음과 같다.

(가) 중력식 용벽
용벽 자체의 무게로 토압 등의 외력을 지지하여 자중으로 토압에 저항하는 형식으로 <그림 4>와 같다.

(나) 반중력식 용벽
반중력식 용벽은 중력식 용벽의 벽두께를 얇게 하고 이로 인해 생기는 인장 응력에 저항하기 위해 철근을 배치한 형식으로 <그림 5>와 같다.

(다) 캔틸레버(Cantilever)식 용벽

① 역T형 용벽: 용벽의 배면에 기초 슬래브가 일부 돌출한 모양의 용벽형식으로 <그림 6>과 같다.

② L형 용벽: L형 용벽은 한쪽 끝이 고정되고 다른 끝은 받쳐지지 않은 상태로 되어있는 보를 이용해 용벽의 재료를 절약하는 형식으로 <그림 7>과 같다.
